# Chapter 5. Recurrent Neural Network (RNN)

Word2vec 만으로는 자연어 처리가 원만하게 이루어질 수 없다! 

- CBOW (continuous bag-of-words)는 주변 단어를 순서에 무관하게 고려하여 학습하는 단어 분산 모델.
- Language model (언어 모델) : 단어의 시퀀스가 일어날 가능성이 어느 정도인지 확률로 평가.<br>
m개의 단어 시퀀스 $\{w_1, w_2, \cdots, w_m\}$에 대하여
\begin{equation}
P(w_1, w_2, \cdots, w_m) = \prod^{m}_{t=1}{P(w_t|w_1,\cdots,w_{t-1})} .
\end{equation}

- 위처럼, 단어의 시퀀스에 따라 확률이 정의되기 때문에, 이를 고려한 확률 모델을 형성할 필요가 있음.

## 1. RNN, 순환신경망

- 시계열 데이터 $(x_0, x_1, \cdots, x_t, \cdots)$ 를 학습하는 데 용이한 모델. ($t$ : 시간?)
- $x_t$ -> RNN layer -> $h_t$, $h_t$가 layer에 다시 들어가는 구조.
\begin{equation}
h_t = \tanh{(h_{t-1} W_h + x_t W_x + b)} .
\end{equation}

- 위 식과 같이, Input에 대응하는 weight $W_x$와 이전 output에 대응하는 $W_h$가 존재함.
- $h_t$는 다음 계층으로 넘어감과 동시에 다음 시각 $t+1$의 RNN 계층으로도 넘어간다.
- $h_{t-1}$ 상태에서 $h_t$로 갱신(업데이트)된다고 볼 수 있음. (Memory?)
- $h$ : hidden state, or hidden state vector. 

<hr>

### (1) BPTT (BackPropagation Through Time)

- BPTT : RNN의 경우, 시간에 따라 RNN 계층으로 이전 시간의 값이 들어오기 때문에, <br>
    이 흐름을 따라서도 역전파법으로 파라미터 (weight, bias)를 학습할 수 있다.  
- 하지만, 긴 시계열의 경우
    - 계산량이 너무 많아지게 되는 문제가 생긴다.
    - 역전파시의 기울기가 불안정해지는 문제가 생긴다. (Vanishing gradient, Overflowing, ...)

### (2) Truncated BPTT

- 다루기 힘든 큰 시계열의 경우 시간축으로 너무 길어진 신경망을 적당한 길이에서 잘라내어 역전파법을 수행.
- Example : $x_0$ ~ $x_9$, $x_10$ ~ $x_19$ 식으로 10개 단위로 끊어서 역전파 시행. (RNN이 10개 단위로 학습하게끔 함.)
    - cf 1) 순전파는 그대로 진행함에 유의할 것.
    - cf 2) 시계열 데이터 $(x_0, x_1, \cdots, x_t, \cdots)$ 는 시간 순서대로 입력되어야 함. 
      (mini-batch 학습은 무작위였음에 유의.)
- 위의 예시에 대한 학습 순서 : $x_0$ ~ $x_9$ 에 대해 순전파 및 역전파 시행 ->
    $h_9$를 입력받아, $x_{10}$ ~ $x_{19}$ 에 대해 순전파 및 역전파 시행. ($h_9$에 대한 기울기는 계산 X) -> ...
    
<hr>

### (3) Mini-batch 학습
- Mini-batch는 무작위로 뽑되, 연속된 데이터 셋을 뽑는다. 
(ex: 500번째 데이터부터 시작할 경우, ($x_{500}$, $x_{501}$, $\cdots$) ) 
- 각 batch 별로 데이터 제공 시작 위치를 옮겨줘야 함. (1번째가 $x_0$ -> $x_{10}$ 이면 2번째는 $x_{500}$ -> $x_{510}$.)

## 2. RNN 구현

$x_s = (x_0, x_1, \cdots, x_{T-1})$ 의 시계열 입력값들에 대한 출력 $h_s=(h_0,h_1,\cdots,h_{T-1})$과 같은 구조를
하나의 계층이라고 하자.

- $N$ : mini-batch size
- $D$ : Input dimension
- $H$ : layer neuron size
- $W_h$, $W_x$ : weight matrix ($h$와 $x$ 각각에 대한)
- $ \tanh{(h_{t-1} W_h + x_{t} W_x + b)} = h_t $ 에 대하여
    - $h_{t-1}$, $h_t$ : $N \times H$ matrix
    - $W_h$ : $H \times H$, square matrix (!!)
    - $W_x$ : $N \times D$
    - $b$ : $1 \times H$ 인데... 각 배치별로 더해주는 거니까, $N \times H$로 생각하자 (같은 행이 N개 나열된 구조). cf) Repeat 노드, 1.3.4 절 참고

In [ ]:
# 단일 RNN class : 이전 time의 값을 받아 1회 순전파 및 역전파를 하는 class.
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    # Forward propagation
    # h_prev, 전 시간대의 입력을 받는다.
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(x,Wx) + np.matmul(h_prev,Wh) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    # backward propagation, 이후 시간대에 대한 기울기를 입력받음.
    # Output : Input 및 h_prev에 대한 기울기를 반환 (이전 layers로 backpropagation을 위해 넘겨줌)
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2) # tanh의 미분
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev